In [1]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# Load player history (training data)
# df_train = pd.read_csv('data/during-season/player_history.csv', parse_dates=['kickoff_time'])
df_train = pd.read_csv('data/during-season/player_history.csv')

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2169 entries, 0 to 2168
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   element                          2169 non-null   int64 
 1   fixture                          2169 non-null   int64 
 2   opponent_team                    2169 non-null   object
 3   total_points                     2169 non-null   int64 
 4   was_home                         2169 non-null   bool  
 5   kickoff_time                     2169 non-null   object
 6   team_h_score                     2169 non-null   int64 
 7   team_a_score                     2169 non-null   int64 
 8   round                            2169 non-null   int64 
 9   minutes                          2169 non-null   int64 
 10  goals_scored                     2169 non-null   int64 
 11  assists                          2169 non-null   int64 
 12  clean_sheets                     2

In [4]:
df_train.head(5)

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,recoveries,value,transfers_balance,selected,transfers_in,transfers_out,name,team_id,team,opponent_team_id
0,1,6,Kalmar FF,6,False,2022-04-03T15:30:00Z,0,1,1,90,...,5,60,0,4920,0,0,Johan Dahlin,1,Malmö FF,10
1,1,16,IF Elfsborg,3,True,2022-04-11T17:10:00Z,1,1,2,90,...,6,60,201,5787,284,83,Johan Dahlin,1,Malmö FF,5
2,1,24,AIK,7,True,2022-04-17T13:00:00Z,3,0,3,90,...,11,60,39,5997,127,88,Johan Dahlin,1,Malmö FF,12
3,1,32,IFK Värnamo,7,False,2022-04-21T17:00:00Z,0,0,4,90,...,5,60,241,6355,319,78,Johan Dahlin,1,Malmö FF,18
4,1,39,IFK Göteborg,6,True,2022-04-25T17:10:00Z,1,0,5,90,...,6,60,144,6527,244,100,Johan Dahlin,1,Malmö FF,4


In [5]:
df_tmp_X = df_train[['round', 'name', 'team', 'opponent_team', 'was_home']]
train_y = df_train[['total_points']]

In [6]:
df_tmp_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2169 entries, 0 to 2168
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   round          2169 non-null   int64 
 1   name           2169 non-null   object
 2   team           2169 non-null   object
 3   opponent_team  2169 non-null   object
 4   was_home       2169 non-null   bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 70.0+ KB


In [7]:
df_tmp_X.head()

,round,name,team,opponent_team,was_home
0,1,Johan Dahlin,Malmö FF,Kalmar FF,False
1,2,Johan Dahlin,Malmö FF,IF Elfsborg,True
2,3,Johan Dahlin,Malmö FF,AIK,True
3,4,Johan Dahlin,Malmö FF,IFK Värnamo,False
4,5,Johan Dahlin,Malmö FF,IFK Göteborg,True


## Preprocessing Data

In [8]:
df_tmp_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2169 entries, 0 to 2168
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   round          2169 non-null   int64 
 1   name           2169 non-null   object
 2   team           2169 non-null   object
 3   opponent_team  2169 non-null   object
 4   was_home       2169 non-null   bool  
dtypes: bool(1), int64(1), object(3)
memory usage: 70.0+ KB


In [9]:
def preprocess_col_gw(df):
    # Make column a category
    df['round'] = df['round'].astype('category')
    
    # Let the column know the range and order of the category
    ordered_gws = list(range(1,31))
    df['round'] = df['round'].cat.set_categories(ordered_gws, ordered=True)
    return df

preprocess_col_gw(df_tmp_X)

<ipython-input-9-4dc78b292f67>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round'] = df['round'].astype('category')
<ipython-input-9-4dc78b292f67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round'] = df['round'].cat.set_categories(ordered_gws, ordered=True)


,round,name,team,opponent_team,was_home
0,1,Johan Dahlin,Malmö FF,Kalmar FF,False
1,2,Johan Dahlin,Malmö FF,IF Elfsborg,True
2,3,Johan Dahlin,Malmö FF,AIK,True
3,4,Johan Dahlin,Malmö FF,IFK Värnamo,False
4,5,Johan Dahlin,Malmö FF,IFK Göteborg,True
...,...,...,...,...,...
2164,4,Filip Trpchevski,BK Häcken,IFK Norrköping,False
2165,5,Filip Trpchevski,BK Häcken,Helsingborgs IF,False
2166,3,Darrell Tibell,IFK Norrköping,Djurgården,False
2167,4,Darrell Tibell,IFK Norrköping,BK Häcken,True


In [10]:
df_tmp_X['round']

0       1
1       2
2       3
3       4
4       5
       ..
2164    4
2165    5
2166    3
2167    4
2168    5
Name: round, Length: 2169, dtype: category
Categories (30, int64): [1 < 2 < 3 < 4 ... 27 < 28 < 29 < 30]

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

#df_tmp_X['round'] = le.fit_transform(df_tmp_X['round'])

In [12]:
# We need to encode the strings for the scikit model to be able to handle them
# Label encoding for ordered data eg. Low, Medium, High
# OneHot encoding for unordered data
# https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

def preprocess(df):    
    categorical_features = ["name", "team", "opponent_team"]
    one_hot = OneHotEncoder()
    transformer = ColumnTransformer([("one_hot",
                                      one_hot,
                                     categorical_features)],
                                     remainder="passthrough")

    transformed = transformer.fit_transform(df).toarray()
    transformed = pd.DataFrame(transformed)
    return transformed

transformed_X = preprocess(df_tmp_X)
transformed_X

/usr/local/Caskroom/miniconda/base/envs/env/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


,0,1,2,3,4,5,6,7,8,9,...,458,459,460,461,462,463,464,465,466,467
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
2166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
2167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0


In [13]:
# Check number of unique values in columns, just to compare against transformed df
print(df_tmp_X.apply(lambda col: col.nunique()))

round              5
name             434
team              16
opponent_team     16
was_home           2
dtype: int64


In [14]:
# Ok, appearantly, there is an easier way to do that.. but we will skip this for now
dummies = pd.get_dummies(df_tmp_X[["name", "team", "opponent_team"]])
dummies

,name_Abbe Khalili,name_Abdelrahman Saidi,name_Abdul Malik Abubakari,name_Abdussalam Magashy,name_Adam Bergmark Wiberg,name_Adam Carlén,name_Adam Hellborg,name_Adam Ingi Benediktsson,name_Adam Kaied,name_Adam Petersson,...,opponent_team_Helsingborgs IF,opponent_team_IF Elfsborg,opponent_team_IFK Göteborg,opponent_team_IFK Norrköping,opponent_team_IFK Värnamo,opponent_team_IK Sirius,opponent_team_Kalmar FF,opponent_team_Malmö FF,opponent_team_Mjällby AIF,opponent_team_Varbergs BoIS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2165,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2166,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Convert strings into pandas category codes since the regressor model cannot handle strings
# for label, content in df_tmp.items():
    # if pd.api.types.is_string_dtype(content):
        # df_tmp[label] = content.astype("category")
#         df_tmp[label] = pd.Categorical(content).codes

In [16]:
# transformed.to_csv('data/during-season/df_transformed.csv', index=False)

In [17]:
# df_tmp = pd.read_csv('data/during-season/df_tmp.csv')

## Build model

In [18]:
train_X = transformed_X

In [19]:
%%time
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1, random_state=1234)
model.fit(train_X, train_y.values.ravel())

CPU times: user 45.2 s, sys: 248 ms, total: 45.5 s
Wall time: 7.02 s


RandomForestRegressor(n_jobs=-1, random_state=1234)

In [20]:
def plot_feature_importance(columns, importances):
    df = (pd.DataFrame({'features': columns, 'feature_importances': importances})
          .sort_values('feature_importances', ascending=False))
    
    fig, ax = plt.subplots()
    ax.barh(df['features'], df['feature_importances'])
    ax.set_ylabel("Features")
    ax.set_xlabel("Feature importance")
    ax.invert_yaxis()

In [21]:
# plot_feature_importance(df_tmp_X.columns, model.feature_importances_)

In [22]:
df_tmp_X.shape

(2169, 5)

In [23]:
df_tmp_X.sample(5)

,round,name,team,opponent_team,was_home
1393,4,Christoffer Nyman,IFK Norrköping,BK Häcken,True
1674,5,Gideon Mensah,Varbergs BoIS,Mjällby AIF,True
616,2,Gustav Ludwigson,Hammarby,GIF Sundsvall,False
1220,1,Haris Radetinac,Djurgården,Degerfors IF,True
45,1,Markus Björkqvist,Malmö FF,Kalmar FF,False


In [24]:
df_tmp_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2169 entries, 0 to 2168
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   round          2169 non-null   category
 1   name           2169 non-null   object  
 2   team           2169 non-null   object  
 3   opponent_team  2169 non-null   object  
 4   was_home       2169 non-null   bool    
dtypes: bool(1), category(1), object(3)
memory usage: 56.5+ KB


In [25]:
import numpy as np
import scipy as sp
from scipy.cluster import hierarchy as hc
import seaborn as sns

def cluster_columns(df, figsize=(10,6), font_size=12):
    corr = sp.stats.spearmanr(df).correlation
    
    fix, ax = plt.subplots(figsize=(10, 15))
    ax = sns.heatmap(corr, annot=True, linewidths=0.5, fmt='.3f', cmap='YlGnBu')
    plt.show()
    
# cluster_columns(df_tmp_X)

In [26]:
from scipy.cluster.hierarchy import dendrogram, linkage

def cluster_columns(df):
    print(df.shape)
    linked = linkage(df)
    print(linked.shape)
    
    
# cluster_columns(df_tmp_X)

In [27]:
import numpy as np
from scipy.cluster import hierarchy as hc
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram


def cluster_columns(df, figsize=(20,16), font_size=12):
    corr = np.round(sp.stats.spearmanr(df).correlation, 4)
    corr_condensed = hc.distance.squareform(1-corr)
    z = hc.linkage(corr_condensed, method='average')
    fig = plt.figure(figsize=figsize)
    hc.dendrogram(z, labels=df.columns, orientation='left', leaf_font_size=font_size)
    plt.show()
    
# cluster_columns(df_tmp_X)

## Predict

In [28]:
# Show train data to see correct format
df_tmp_X.head()

,round,name,team,opponent_team,was_home
0,1,Johan Dahlin,Malmö FF,Kalmar FF,False
1,2,Johan Dahlin,Malmö FF,IF Elfsborg,True
2,3,Johan Dahlin,Malmö FF,AIK,True
3,4,Johan Dahlin,Malmö FF,IFK Värnamo,False
4,5,Johan Dahlin,Malmö FF,IFK Göteborg,True


### Get data to predict

In [29]:
df_all_player_fixtures = pd.read_csv('data/during-season/player_fixtures.csv')

In [30]:
# Next gameweek to predict
gw_to_predict = df_all_player_fixtures['event'].min()

In [31]:
df_predict = df_all_player_fixtures[['event', 'name', 'team', 'opponent_team', 'is_home']]
df_predict = df_predict.rename(columns={'event': 'round', 'is_home': 'was_home'})
df_predict = df_predict[df_predict['round'] == gw_to_predict]
df_predict.head(20)

,round,name,team,opponent_team,was_home
0,6,Johan Dahlin,Malmö FF,Hammarby,False
25,6,Ismael Diawara,Malmö FF,Hammarby,False
50,6,Niklas Moisander,Malmö FF,Hammarby,False
75,6,Jonas Knudsen,Malmö FF,Hammarby,False
100,6,Lasse Nielsen,Malmö FF,Hammarby,False
125,6,Eric Larsson,Malmö FF,Hammarby,False
150,6,Felix Beijmo,Malmö FF,Hammarby,False
175,6,Martin Olsson,Malmö FF,Hammarby,False
200,6,Matej Chalus,Malmö FF,Hammarby,False
225,6,Markus Björkqvist,Malmö FF,Hammarby,False


In [32]:
preprocess_col_gw(df_predict)
df_predict['round']

0        6
25       6
50       6
75       6
100      6
        ..
10750    6
10775    6
10800    6
10825    6
10850    6
Name: round, Length: 492, dtype: category
Categories (30, int64): [1 < 2 < 3 < 4 ... 27 < 28 < 29 < 30]

In [33]:
df_predict_transformed = preprocess(df_predict)

/usr/local/Caskroom/miniconda/base/envs/env/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


### Make prediction

In [34]:
predicted = model.predict(df_predict_transformed)

In [35]:
df_predicted = pd.DataFrame(predicted)

In [36]:
df_predict.reset_index(drop=True, inplace=True)
df_predicted.reset_index(drop=True, inplace=True)
df_result = df_predict.join(df_predicted)
df_result.rename(columns={0: 'predicted_points'}, inplace=True)
df_result.sort_values(by='predicted_points', ascending=False, inplace=True)

# Print the top predicted points for round
df_result.head(20)

,round,name,team,opponent_team,was_home,predicted_points
141,6,Mohanad Jeahze,Hammarby,Malmö FF,True,7.79
154,6,Nahir Besara,Hammarby,Malmö FF,True,7.47
139,6,Simon Sandberg,Hammarby,Malmö FF,True,6.98
149,6,Williot Swedberg,Hammarby,Malmö FF,True,6.94
415,6,Carlos Moros Gracia,Mjällby AIF,Helsingborgs IF,True,6.84
410,6,Noah Eile,Mjällby AIF,Helsingborgs IF,True,6.82
409,6,Samuel Brolin,Mjällby AIF,Helsingborgs IF,True,6.62
4,6,Lasse Nielsen,Malmö FF,Hammarby,False,6.61
323,6,Jonathan Levi,IFK Norrköping,IFK Värnamo,True,6.59
151,6,Gustav Ludwigson,Hammarby,Malmö FF,True,6.44


In [37]:
df_result.tail()

,round,name,team,opponent_team,was_home,predicted_points
201,6,Gustav Lillienberg,IFK Göteborg,Kalmar FF,True,0.00
202,6,Hákon Rafn Valdimarsson,IF Elfsborg,Degerfors IF,False,0.00
394,6,Victor Karlsson,Varbergs BoIS,BK Häcken,False,0.00
420,6,Adam Petersson,Mjällby AIF,Helsingborgs IF,True,0.00
31,6,Forrest Lasso,GIF Sundsvall,AIK,True,-0.02


### Add "predicted point/cost"

In [38]:
# Get latest price for every player
player_last_gw = df_train.copy()
last_gw_indices = player_last_gw.groupby('name')['round'].agg(pd.Series.idxmax)
player_last_gw = player_last_gw.iloc[last_gw_indices]
player_last_gw.set_index('name', inplace=True)
df_result['price (M)'] = df_result['name'].map(player_last_gw['value']/10)

# Calculate points/price
df_result['predicted_point_per_price'] = df_result['predicted_points']/df_result['price (M)']
df_result.sort_values(by=['predicted_point_per_price'], ascending=False, inplace=True)
df_result.head(30)

,round,name,team,opponent_team,was_home,predicted_points,price (M),predicted_point_per_price
410,6,Noah Eile,Mjällby AIF,Helsingborgs IF,True,6.82,4.8,1.420833
139,6,Simon Sandberg,Hammarby,Malmö FF,True,6.98,5.0,1.396000
418,6,Noah Persson,Mjällby AIF,Helsingborgs IF,True,5.99,4.5,1.331111
409,6,Samuel Brolin,Mjällby AIF,Helsingborgs IF,True,6.62,5.0,1.324000
92,6,Charlie Weberg,Helsingborgs IF,Mjällby AIF,False,5.90,4.5,1.311111
93,6,Charlie Weberg,Helsingborgs IF,Djurgården,False,5.90,4.5,1.311111
413,6,Ivan Kricak,Mjällby AIF,Helsingborgs IF,True,5.65,4.5,1.255556
415,6,Carlos Moros Gracia,Mjällby AIF,Helsingborgs IF,True,6.84,5.5,1.243636
33,6,Alexander Blomqvist,GIF Sundsvall,AIK,True,6.18,5.0,1.236000
149,6,Williot Swedberg,Hammarby,Malmö FF,True,6.94,5.8,1.196552
